In [ ]:
print("Compare real distance vs Pelni's")

In [1]:
import pandas.io.sql as sqlio
import psycopg2 as ps
import os
from dotenv import load_dotenv

In [2]:
import nautical_calculations
from nautical_calculations.basic import get_distance
from nautical_calculations.operations import convert_to_miles
import numpy as np
from geopy.distance import geodesic as gd
import math
import haversine as hs
from haversine import Unit

In [3]:
load_dotenv()
db = os.getenv('POSTGRES_DB')
user = os.getenv('POSTGRES_USER')
password = os.getenv('POSTGRES_PASSWORD')
host = os.getenv('POSTGRES_SERVER')
port = os.getenv('POSTGRES_PORT')

In [4]:
conn = ps.connect(dbname=db, 
                  user=user,
                  password=password, 
                  host=host,
                  port=port)

In [5]:
# accommodate query
sql = """SELECT jarak_nautis as nautical,mp2.id_port as port_destination,mp.id_port as port_origin,mp.code as code_origin,mp2.code as code_destination,
mp.latitude as lat_origin, mp.longitude as long_origin,mp2.latitude as lat_dest, mp2.longitude as long_dest,
mp.port_name as name_origin, mp2.port_name as name_dest
FROM stg_pelni.temp_jarak tj  
LEFT JOIN stg_pelni.master_port mp on tj.pelabuhan_asal=mp.port_name
LEFT JOIN stg_pelni.master_port mp2 on tj.pelabuhan_tujuan=mp2.port_name

WHERE mp.id_port is not null AND mp2.id_port is not null;"""

In [6]:
# Read Query
df = sqlio.read_sql_query(sql, conn)
df

C:\Users\BN001602001\AppData\Local\Temp\ipykernel_3692\2093454835.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = sqlio.read_sql_query(sql, conn)


,nautical,port_destination,port_origin,code_origin,code_destination,lat_origin,long_origin,lat_dest,long_dest,name_origin,name_dest
0,477,00739854-654f-469c-b5af-31680f7ae36e,19887a24-9011-425f-bc0c-5f5cd512ba5a,563,662,-7.196403,112.73302,-8.493639,119.87533,Tanjung Perak,Labuan Bajo
1,104,00739854-654f-469c-b5af-31680f7ae36e,7fce1b97-29ae-42e7-ac8f-4523753e527b,685,662,-9.637536,120.25076,-8.493639,119.87533,Waingapu,Labuan Bajo
2,260,b456ed3f-543d-416e-98a2-e835bf4f4bec,cc0756e0-437b-4403-b4c9-14d4bc895620,861,920,1.050458,120.80008,None,None,Toli-toli,Amurang
3,75,13ae6ff9-3a38-42f4-bf51-ac03c0492d30,b2811ee3-48f4-41a3-b82a-bcd88ef3233c,946,952,-3.693357,128.1766,-3.338247,128.91997,Ambon,Amahai
4,65,0c87bc40-0218-4bc1-bbd8-3dfff430a3d9,ec8ec4a4-710f-410f-b2ad-7802b6bf0a39,844,845,3.940083,126.69485,4.722646,127.08841,Lirung,Karatung
...,...,...,...,...,...,...,...,...,...,...,...
143,380,3bd1c07f-44b1-4fb8-becc-e6faa9882fb3,f757a369-8a39-4910-a47f-68dad517d1fb,948,979,-5.522391,138.13821,-8.477812,140.3904,Agats,Merauke
144,146,72346087-6a6a-462b-a434-913d6f2e5ca9,7d09ed71-36f8-40bf-996a-5a09bae5dc3d,835,942,1.440998,125.20018,0.780123,127.38839,Bitung,Ternate
145,323,72346087-6a6a-462b-a434-913d6f2e5ca9,b2811ee3-48f4-41a3-b82a-bcd88ef3233c,946,942,-3.693357,128.1766,0.780123,127.38839,Ambon,Ternate
146,170,1334ec95-cc8a-4464-a263-0b0beee559da,ddac0441-65ff-4960-b993-87c73ebc2d85,322,251,-1.62999,105.733,0.860583,104.6115,Belinyu,Sei Kolak Kijang


In [7]:
# Cleansing from data status 'None'
cleandf = df[~((df['lat_origin']=='None') | (df['long_origin']=='None') | (df['lat_dest']=='None') | (df['long_dest']=='None'))]
cleandf


,nautical,port_destination,port_origin,code_origin,code_destination,lat_origin,long_origin,lat_dest,long_dest,name_origin,name_dest
0,477,00739854-654f-469c-b5af-31680f7ae36e,19887a24-9011-425f-bc0c-5f5cd512ba5a,563,662,-7.196403,112.73302,-8.493639,119.87533,Tanjung Perak,Labuan Bajo
1,104,00739854-654f-469c-b5af-31680f7ae36e,7fce1b97-29ae-42e7-ac8f-4523753e527b,685,662,-9.637536,120.25076,-8.493639,119.87533,Waingapu,Labuan Bajo
2,260,b456ed3f-543d-416e-98a2-e835bf4f4bec,cc0756e0-437b-4403-b4c9-14d4bc895620,861,920,1.050458,120.80008,None,None,Toli-toli,Amurang
3,75,13ae6ff9-3a38-42f4-bf51-ac03c0492d30,b2811ee3-48f4-41a3-b82a-bcd88ef3233c,946,952,-3.693357,128.1766,-3.338247,128.91997,Ambon,Amahai
4,65,0c87bc40-0218-4bc1-bbd8-3dfff430a3d9,ec8ec4a4-710f-410f-b2ad-7802b6bf0a39,844,845,3.940083,126.69485,4.722646,127.08841,Lirung,Karatung
...,...,...,...,...,...,...,...,...,...,...,...
142,95,c069f8c4-4a93-4021-9a75-c9aadb3b4a1c,13ae6ff9-3a38-42f4-bf51-ac03c0492d30,952,947,-3.338247,128.91997,-4.524874,129.8967,Amahai,Banda Naira
143,380,3bd1c07f-44b1-4fb8-becc-e6faa9882fb3,f757a369-8a39-4910-a47f-68dad517d1fb,948,979,-5.522391,138.13821,-8.477812,140.3904,Agats,Merauke
144,146,72346087-6a6a-462b-a434-913d6f2e5ca9,7d09ed71-36f8-40bf-996a-5a09bae5dc3d,835,942,1.440998,125.20018,0.780123,127.38839,Bitung,Ternate
145,323,72346087-6a6a-462b-a434-913d6f2e5ca9,b2811ee3-48f4-41a3-b82a-bcd88ef3233c,946,942,-3.693357,128.1766,0.780123,127.38839,Ambon,Ternate


In [8]:
cleandf = cleandf.dropna(subset=['lat_origin'])
cleandf = cleandf.dropna(subset=['long_origin'])

cleandf = cleandf.dropna(subset=['lat_dest'])
cleandf = cleandf.dropna(subset=['long_dest'])

In [9]:
#rename column
cleandf = cleandf.rename(columns={'nautical': 'nautical_pelni'})

In [10]:
#change typedata
cleandf['lat_origin'] = cleandf['lat_origin'].astype(float)
cleandf['lat_dest'] = cleandf['lat_dest'].astype(float)
cleandf['long_origin'] = cleandf['long_origin'].astype(float)
cleandf['long_dest'] = cleandf['long_dest'].astype(float)

# Checked typedata
cleandf.dtypes


nautical_pelni        int64
port_destination     object
port_origin          object
code_origin          object
code_destination     object
lat_origin          float64
long_origin         float64
lat_dest            float64
long_dest           float64
name_origin          object
name_dest            object
dtype: object

In [11]:
# Calculate from latitude and longtitude data convert to miles
# Looping data
nautical_data1 = []
for index, row in cleandf.iterrows():
        result = convert_to_miles(get_distance( row['lat_origin'],row['long_origin'],row['lat_dest'],row['long_dest']))
        nautical_data1.append(result)

cleandf['nautical_python_1'] = nautical_data1
        

In [12]:
# Calculate from latitude and longtitude data convert to miles (use other library)
nautical_data2 = []
for index, row in cleandf.iterrows():
    ori = (row['lat_origin'],row['long_origin'])
    des = (row['lat_dest'],row['long_dest'])
    rs = gd(ori, des).miles
    nautical_data2.append(rs)
    
cleandf['nautical_python_2'] = nautical_data2    
    

In [13]:
# Calculate from latitude and longtitude data convert to miles (use other library - Math)

def haversine_distance(lat1,long1, lat2, long2):
    # Convert Latitude and longitude from degress to radians
    lat1_rad = math.radians(lat1)
    long1_rad = math.radians(long1)
    lat2_rad = math.radians(lat2)
    long2_rad = math.radians(long2)
    
    # Haversine formula
    dlong = long2_rad - long1_rad
    dlat = lat2_rad - lat1_rad
    a = math.sin(dlat/2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlong/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    
    # Earth radius in nautical miles (approximately 3440.1 NM)
    r = 3440.1
    
    # Calculate the nautical distance
    distance_nm = r * c
    return distance_nm

In [14]:
# calculate from latitude and longtitude data convert to miles (use other - Haversine)
nautical_data3 = []
for index, row in cleandf.iterrows():
    lat1 = row['lat_origin']
    long1 = row['long_origin']
    lat2 = row['lat_dest']
    long2 = row['long_dest']
    distance_nm = haversine_distance(lat1, long1, lat2, long2)
    nautical_data3.append(distance_nm)
    
cleandf['nautical_python_3'] = nautical_data3   
    

In [15]:
# calculate from latitude and longtitude data convert to miles (use other library - MPU)
nautical_data4 = []
for index, row in cleandf.iterrows():
    ori = (row['lat_origin'],row['long_origin'])
    des = (row['lat_dest'],row['long_dest'])
    
    # Calculate distance in miles 
    dist = hs.haversine(ori,des, unit=Unit.MILES)
    nautical_data4.append(dist)

cleandf['nautical_python_4'] = nautical_data4   

In [16]:
# Change Typedata
cleandf['nautical_pelni'] = cleandf['nautical_pelni'].astype(float)

In [17]:
# Calculate percentase (nautical_pelni v nautical_python)
persentase1 = []
for index, row in cleandf.iterrows():
    ps = row['nautical_python_1'] - row['nautical_pelni']
    ps = ps / row['nautical_pelni'] * 100
    ps = round(ps, 1)
    persentase1.append(ps)
    
    
cleandf['persentase_python_1'] = persentase1

In [18]:
# Calculate percentase (nautical_pelni v nautical_python)
persentase2 = []
for index, row in cleandf.iterrows():
    ps = row['nautical_python_2'] - row['nautical_pelni']
    ps = ps / row['nautical_pelni'] * 100
    ps = round(ps, 1)
    persentase2.append(ps)
    
    
cleandf['persentase_python_2'] = persentase2

In [19]:
# Calculate percentase (nautical_pelni v nautical_python)
persentase3 = []
for index, row in cleandf.iterrows():
    ps = row['nautical_python_3'] - row['nautical_pelni']
    ps = ps / row['nautical_pelni'] * 100
    ps = round(ps, 1)
    persentase3.append(ps)
    
    
cleandf['persentase_python_3'] = persentase3

In [20]:
# Calculate percentase (nautical_pelni v nautical_python)
persentase4 = []
for index, row in cleandf.iterrows():
    ps = row['nautical_python_4'] - row['nautical_pelni']
    ps = ps / row['nautical_pelni'] * 100
    ps = round(ps, 1)
    persentase4.append(ps)
    
    
cleandf['persentase_python_4'] = persentase4

In [21]:
# Change Number Negative to positive
cleandf['persentase_python_1'] = abs(cleandf['persentase_python_1'])
cleandf['persentase_python_2'] = abs(cleandf['persentase_python_2'])
cleandf['persentase_python_3'] = abs(cleandf['persentase_python_3'])
cleandf['persentase_python_4'] = abs(cleandf['persentase_python_4'])

In [24]:
cleandf.head()

,nautical_pelni,port_destination,port_origin,code_origin,code_destination,lat_origin,long_origin,lat_dest,long_dest,name_origin,name_dest,nautical_python_1,nautical_python_2,nautical_python_3,nautical_python_4,persentase_python_1,persentase_python_2,persentase_python_3,persentase_python_4
0,477.0,00739854-654f-469c-b5af-31680f7ae36e,19887a24-9011-425f-bc0c-5f5cd512ba5a,563,662,-7.196403,112.73302,-8.493639,119.87533,Tanjung Perak,Labuan Bajo,496.999161,497.481177,431.885017,497.000002,4.2,4.3,9.5,4.2
1,104.0,00739854-654f-469c-b5af-31680f7ae36e,7fce1b97-29ae-42e7-ac8f-4523753e527b,685,662,-9.637536,120.25076,-8.493639,119.87533,Waingapu,Labuan Bajo,83.082903,82.691595,72.197830,83.083044,20.1,20.5,30.6,20.1
3,75.0,13ae6ff9-3a38-42f4-bf51-ac03c0492d30,b2811ee3-48f4-41a3-b82a-bcd88ef3233c,946,952,-3.693357,128.17660,-3.338247,128.91997,Ambon,Amahai,56.834109,56.827980,49.388011,56.834205,24.2,24.2,34.1,24.2
4,65.0,0c87bc40-0218-4bc1-bbd8-3dfff430a3d9,ec8ec4a4-710f-410f-b2ad-7802b6bf0a39,844,845,3.940083,126.69485,4.722646,127.08841,Lirung,Karatung,60.487562,60.234628,52.562809,60.487664,6.9,7.3,19.1,6.9
6,110.0,9c1e061d-9b9f-404d-a4cc-c98db3337a64,e5efe83c-6d0c-4b8c-8025-84ade2e12366,928,951,-2.056930,125.98209,-3.269646,127.08712,Sanana,Namlea,113.302253,113.016292,98.458004,113.302444,3.0,2.7,10.5,3.0


In [22]:
# Sum number percentage

print(cleandf['persentase_python_1'].sum())
print(cleandf['persentase_python_2'].sum())
print(cleandf['persentase_python_3'].sum())
print(cleandf['persentase_python_4'].sum())


1940.3000000000002
1932.2999999999997
2528.6
1940.3000000000002


In [ ]:
#export to CSV
# cleandf.to_csv('\data/compare_distance.csv')

In [ ]:
cleandf.columns.values

In [25]:
fd = cleandf.reindex(columns=['port_destination', 'port_origin', 'code_origin', 'code_destination', 'name_origin', 'name_dest', 'nautical_python_2'])
fd

,port_destination,port_origin,code_origin,code_destination,name_origin,name_dest,nautical_python_2
0,00739854-654f-469c-b5af-31680f7ae36e,19887a24-9011-425f-bc0c-5f5cd512ba5a,563,662,Tanjung Perak,Labuan Bajo,497.481177
1,00739854-654f-469c-b5af-31680f7ae36e,7fce1b97-29ae-42e7-ac8f-4523753e527b,685,662,Waingapu,Labuan Bajo,82.691595
3,13ae6ff9-3a38-42f4-bf51-ac03c0492d30,b2811ee3-48f4-41a3-b82a-bcd88ef3233c,946,952,Ambon,Amahai,56.827980
4,0c87bc40-0218-4bc1-bbd8-3dfff430a3d9,ec8ec4a4-710f-410f-b2ad-7802b6bf0a39,844,845,Lirung,Karatung,60.234628
6,9c1e061d-9b9f-404d-a4cc-c98db3337a64,e5efe83c-6d0c-4b8c-8025-84ade2e12366,928,951,Sanana,Namlea,113.016292
...,...,...,...,...,...,...,...
142,c069f8c4-4a93-4021-9a75-c9aadb3b4a1c,13ae6ff9-3a38-42f4-bf51-ac03c0492d30,952,947,Amahai,Banda Naira,105.786862
143,3bd1c07f-44b1-4fb8-becc-e6faa9882fb3,f757a369-8a39-4910-a47f-68dad517d1fb,948,979,Agats,Merauke,255.247726
144,72346087-6a6a-462b-a434-913d6f2e5ca9,7d09ed71-36f8-40bf-996a-5a09bae5dc3d,835,942,Bitung,Ternate,157.996510
145,72346087-6a6a-462b-a434-913d6f2e5ca9,b2811ee3-48f4-41a3-b82a-bcd88ef3233c,946,942,Ambon,Ternate,312.158550
